<a href="https://colab.research.google.com/github/AschHarwood/social_unrest/blob/main/data_processing_adding_policy_data_11_15_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Installs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)


# Data Processing

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/social_unrest_data/state_social_unrest_10_28 (1).parquet')

In [2]:
main = pd.read_parquet('/content/drive/MyDrive/social_unrest_data/main_dataset_embeddings_10_26 (2).parquet')

In [3]:
orig = pd.read_feather('/content/drive/MyDrive/social_unrest_data/main_data_9_13.feather')

In [4]:
orig_abridged = orig[['state_fips', 'fips','date', 'protest_count']]

In [5]:
orig_abridged.date = pd.to_datetime(orig_abridged.date)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [6]:
orig_abridged = orig_abridged[orig_abridged.fips!='nan']

In [7]:
orig_abridged.state_fips = orig_abridged.state_fips.astype(int)
orig_abridged.fips = orig_abridged.fips.astype(int)

In [8]:
orig_abridged.set_index(['date', 'state_fips', 'fips'], inplace=True)

In [9]:
orig_abridged = orig_abridged.astype(float)

In [10]:
main.set_index(['date', 'state_fips', 'fips'], inplace=True)

In [11]:
merged = main.merge(orig_abridged, how='left', left_index=True, right_index=True)

In [14]:
merged.info(max_cols=500)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1507578 entries, (Timestamp('2020-01-07 00:00:00'), 6, 6067) to (Timestamp('2021-05-24 00:00:00'), 78, 78020)
Data columns (total 320 columns):
 #   Column                                                  Non-Null Count    Dtype  
---  ------                                                  --------------    -----  
 0   cases_avg                                               1507578 non-null  float64
 1   cases_avg_per_100k                                      1507578 non-null  float64
 2   deaths_avg                                              1507578 non-null  float64
 3   deaths_avg_per_100k                                     1507578 non-null  float64
 4   cases                                                   1507578 non-null  float64
 5   deaths                                                  1507578 non-null  float64
 6   Mobility_score                                          1507578 non-null  float64
 7   info_tweets       

# Adding Policy Data

In [12]:
import numpy as np

In [13]:
!git clone https://github.com/KFFData/COVID-19-Data.git

Cloning into 'COVID-19-Data'...
remote: Enumerating objects: 2957, done.
remote: Counting objects: 100% (482/482), done.
remote: Compressing objects: 100% (482/482), done.
remote: Total 2957 (delta 295), reused 0 (delta 0), pack-reused 2475
Receiving objects: 100% (2957/2957), 254.18 MiB | 8.62 MiB/s, done.
Resolving deltas: 100% (1594/1594), done.


In [15]:
import glob
path = r'/content/COVID-19-Data/State Policy Actions/State Social Distancing Actions'
all_files = glob.glob(path+"/*.csv")
df_list = []
for filename in all_files:
  policy = pd.read_csv(filename)

  policy.drop(0, inplace=True)
  date = filename[76:]
  policy.rename(columns={'Unnamed: 0': 'state'}, inplace=True)

  policy['date'] = date
  df_list.append(policy)
complete_policy_actions = pd.concat(df_list)
complete_policy_actions = complete_policy_actions[complete_policy_actions.state!='United States']
import dateutil.parser as dparser
complete_policy_actions.date = np.where(complete_policy_actions.date.values=='20201-06-01_Social_Distancing.csv','2021-06-01', complete_policy_actions.date.values )
complete_policy_actions['date_object'] = complete_policy_actions.date.apply(lambda x: dparser.parse(x, fuzzy=True))
complete_policy_actions.sort_values(by='date_object', inplace=True)

In [17]:
for state in complete_policy_actions.state.unique():
  complete_policy_actions.loc[complete_policy_actions.state==state] = complete_policy_actions.loc[complete_policy_actions.state==state].fillna(method='pad')
for state in complete_policy_actions.state.unique():
  complete_policy_actions.loc[complete_policy_actions.state==state] = complete_policy_actions.loc[complete_policy_actions.state==state].fillna(method='bfill')

In [19]:
%pip install us
import us
name_lookup = us.states.mapping('name', 'fips')
complete_policy_actions['state_fips'] = complete_policy_actions.state.apply(lambda x: name_lookup[x])

     |████████████████████████████████| 132 kB 37.7 MB/s 
  Created wheel for us: filename=us-2.0.2-py3-none-any.whl size=11942 sha256=9afc95654a8f6db88719e87b40dbb4fcf33f0b6a73301b853ff634cf96aa56ba
  Stored in directory: /root/.cache/pip/wheels/ca/6b/11/cda9ea2438f721330a35c9a2c8e34b4aedcd34c89af48a4d00
  Created wheel for jellyfish: filename=jellyfish-0.6.1-cp37-cp37m-linux_x86_64.whl size=72193 sha256=d266c21e25434854dbbf08b8988a01ea82c6acdbdb8127a7c3f15cc2c03159e6
  Stored in directory: /root/.cache/pip/wheels/a1/99/51/7de469e37cd1b3c763c24394e1ebf1baa2d79e094bf346cf80
Successfully built us jellyfish


In [20]:
complete_policy_actions.drop('date', axis=1, inplace=True)
complete_policy_actions.rename(columns={'date_object': 'date'}, inplace=True)

In [ ]:
fips_list = complete_policy_actions.state_fips.unique()
fips_list = list(fips_list)
fips_list

In [23]:
df_list = []
for state in fips_list:
  state_filtered = complete_policy_actions.loc[complete_policy_actions.state_fips==state]
  state_filtered.set_index('date', inplace=True)
  state_name = state_filtered.state.iloc[0]

  missing_dates = pd.date_range(start='2020-01-07', end='2021-04-15').difference(state_filtered.index)
  missing_dates_df = pd.DataFrame(index=missing_dates, columns=state_filtered.columns)
  missing_dates_df.state_fips = state
  missing_dates_df.state = state_name
  state_joined = pd.concat([state_filtered, missing_dates_df])
  state_joined.reset_index(inplace=True)
  state_joined.sort_values(by='index', inplace=True)
  state_joined.fillna(method='ffill', inplace=True)
  state_joined.rename(columns={'index': 'date'}, inplace=True)

  state_joined.fillna('unknown', inplace=True)
  state_joined.set_index('date', inplace=True)
  df_list.append(state_joined)

In [24]:
all_states_policy_actions_revised = pd.concat(df_list)
all_states_policy_actions_revised.reset_index(inplace=True)
all_states_policy_actions_revised.sort_values(by=['date', 'state_fips'],inplace=True)
all_states_policy_actions_revised

,date,state,State Is Easing Social Distancing Measures,Stay at Home Order,Mandatory Quarantine for Travelers,Non-Essential Business Closures,Large Gatherings Ban,School Closures,Restaurant Limits,Primary Election Postponement,Emergency Declaration,Status of Reopening,Bar Closures,Face Covering Requirement,Bar Closures*,Statewide Face Mask Requirement,state_fips
24250,2020-01-07,Alabama,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,01
17945,2020-01-07,Alaska,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,02
10670,2020-01-07,Arizona,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,04
10185,2020-01-07,Arkansas,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,05
9700,2020-01-07,California,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15519,2021-09-21,Virginia,Yes,Lifted,-,All Non-Essential Businesses Open,No Limit,Closed for School Year,Open,-,No,Reopened,Closed,Required for General Public,Open,No,51
15034,2021-09-21,Washington,Yes,Lifted,Lifted,All Non-Essential Businesses Open,No Limit,Closed for School Year,Open,-,Yes,Reopened,Closed,Required for General Public,Open,Indoor Only,53
14549,2021-09-21,West Virginia,Yes,Lifted,Lifted,All Non-Essential Businesses Open,No Limit,Closed for School Year,Open,Postponed,Yes,Reopened,Reopened,Required for General Public,Open,No,54
14064,2021-09-21,Wisconsin,Yes,Lifted,-,All Non-Essential Businesses Open,No Limit,Closed for School Year,Open,-,No,Reopened,Reopened,Required for General Public,Open,No,55


In [25]:
all_states_policy_actions_revised.set_index(['date', 'state_fips'], inplace=True)

In [27]:
merged.reset_index(inplace=True)


In [29]:
merged.state_fips = merged.state_fips.astype(str).str.zfill(2)

In [32]:
all_states_policy_actions_revised

state  \
date       state_fips                  
2020-01-07 01                Alabama   
           02                 Alaska   
           04                Arizona   
           05               Arkansas   
           06             California   
...                              ...   
2021-09-21 51               Virginia   
           53             Washington   
           54          West Virginia   
           55              Wisconsin   
           56                Wyoming   

                      State Is Easing Social Distancing Measures  \
date       state_fips                                              
2020-01-07 01                                            unknown   
           02                                            unknown   
           04                                            unknown   
           05                                            unknown   
           06                                            unknown   
...                                                          ...   
2021-09-21 51                                                Yes   
           53                                                Yes   
           54                                                Yes   
           55                                                Yes   
           56                                                Yes   

                      Stay at Home Order Mandatory Quarantine for Travelers  \
date       state_fips                                                         
2020-01-07 01                    unknown                            unknown   
           02                    unknown                            unknown   
           04                    unknown                            unknown   
           05                    unknown                            unknown   
           06                    unknown                            unknown   
...                                  ...                                ...   
2021-09-21 51                     Lifted                                  -   
           53                     Lifted                             Lifted   
           54                     Lifted                             Lifted   
           55                     Lifted                                  -   
           56                          -                             Lifted   

                         Non-Essential Business Closures Large Gatherings Ban  \
date       state_fips                                                           
2020-01-07 01                                    unknown              unknown   
           02                                    unknown              unknown   
           04                                    unknown              unknown   
           05                                    unknown              unknown   
           06                                    unknown              unknown   
...                                                  ...                  ...   
2021-09-21 51          All Non-Essential Businesses Open             No Limit   
           53          All Non-Essential Businesses Open             No Limit   
           54          All Non-Essential Businesses Open             No Limit   
           55          All Non-Essential Businesses Open             No Limit   
           56          All Non-Essential Businesses Open             No Limit   

                              School Closures Restaurant Limits  \
date       state_fips                                             
2020-01-07 01                         unknown           unknown   
           02                         unknown           unknown   
           04                         unknown           unknown   
           05                         unknown           unknown   
           06                         unknown           unknown   
...                                       ...               ...   
2021-09

In [31]:
merged.set_index(['date', 'state_fips'], inplace=True)
merged

fips  cases_avg  cases_avg_per_100k  deaths_avg  \
date       state_fips                                                     
2020-01-07 06           6067       0.00                0.00        0.00   
           34          34029       0.00                0.00        0.00   
2020-01-08 36          36001       0.00                0.00        0.00   
           08           8031       0.00                0.00        0.00   
2020-01-09 34          34021       0.00                0.00        0.00   
...                      ...        ...                 ...         ...   
2021-05-24 17          17135       3.43               12.07        0.00   
           17          17133       0.71                2.06        0.14   
           17          17131       0.86                5.55        0.00   
           17          17129       0.43                3.51        0.00   
           78          78020       0.86               20.55        0.00   

                       deaths_avg_per_100k  cases  deaths  Mobility_score  \
date       state_fips                                                       
2020-01-07 06                         0.00    0.0     0.0      136.797413   
           34                         0.00    0.0     0.0      136.797413   
2020-01-08 36                         0.00    0.0     0.0      136.797413   
           08                         0.00    0.0     0.0      136.797413   
2020-01-09 34                         0.00    0.0     0.0      136.797413   
...                                    ...    ...     ...             ...   
2021-05-24 17                         0.00    1.0     0.0      136.797413   
           17                         0.41    0.0     0.0      136.797413   
           17                         0.00    1.0     0.0      136.797413   
           17                         0.00    0.0     0.0      136.797413   
           78                         0.00    6.0     0.0      136.797413   

                       info_tweets  info_tweets_T  info_tweets_RE  \
date       state_fips                                               
2020-01-07 06              54427.0        12407.0          1770.0   
           34              54427.0        12407.0          1770.0   
2020-01-08 36              54427.0        12407.0          1770.0   
           08              54427.0        12407.0          1770.0   
2020-01-09 34              54427.0        12407.0          1770.0   
...                            ...            ...             ...   
2021-05-24 17             189186.0        45800.0         20893.0   
           17             189186.0        45800.0         20893.0   
           17             189186.0        45800.0         20893.0   
           17             189186.0        45800.0         20893.0   
           78             189186.0        45800.0         20893.0   

                       info_tweets_RT  info_tweets_local  \
date       state_fips                                      
2020-01-07 06                 40250.0            45957.0   
           34                 40250.0            45957.0   
2020-01-08 36                 40250.0            45957.0   
           08                 40250.0            45957.0   
2020-01-09 34                 40250.0            45957.0   
...                               ...                ...   
2021-05-24 17                122493.0           161652.0   
           17                122493.0           161652.0   
           17                122493.0           161652.0   
           17                122493.0           161652.0   
           78                122493.0           161652.0   

                       info_tweets_verified  info_tweets_unverified  \
date       state_fips                                                 
2020-01-07 06                        2614.0                 51813.0   
           34                        2614.0                 51813.0   
2020-01-08 36                        2614.0                 51813.0   
           0

In [33]:
merged_policy = merged.merge(all_states_policy_actions_revised, how='left', left_index=True, right_index=True)

In [47]:
merged_policy[merged_policy.state.isnull()].state_fips.value_counts()

72    32360
78     1799
69      869
66      130
00        3
Name: state_fips, dtype: int64

In [38]:
merged_policy.reset_index(inplace=True)

In [41]:
for state in merged_policy.state_fips.unique():
  merged_policy.loc[merged_policy.state_fips==state] = merged_policy.loc[merged_policy.state_fips==state].fillna(method='ffill')
  merged_policy.loc[merged_policy.state_fips==state] = merged_policy.loc[merged_policy.state_fips==state].fillna(method='bfill')


In [ ]:
merged_policy.info(max_cols=500)

In [48]:
merged_policy.dropna(inplace=True)

In [51]:
policy_actions = merged_policy.loc[:, 'State Is Easing Social Distancing Measures':]

In [53]:
pa_dummies = pd.get_dummies(policy_actions)

In [54]:
pa_dummies

,State Is Easing Social Distancing Measures_Yes,State Is Easing Social Distancing Measures_unknown,Stay at Home Order_-,Stay at Home Order_Counties of High Transmission (New),Stay at Home Order_During Curfew Hours,Stay at Home Order_High-Risk Groups,Stay at Home Order_Lifted,Stay at Home Order_New Stay at Home Order,Stay at Home Order_Regions of High Transmission,Stay at Home Order_Rolled Back to High Risk Groups,Stay at Home Order_Statewide,Stay at Home Order_Statewide (New),Stay at Home Order_unknown,Mandatory Quarantine for Travelers_-,Mandatory Quarantine for Travelers_All Air Travelers,Mandatory Quarantine for Travelers_All Travelers,Mandatory Quarantine for Travelers_All Travelers (New),Mandatory Quarantine for Travelers_From Certain States,Mandatory Quarantine for Travelers_From Certain States (New),Mandatory Quarantine for Travelers_International Travelers,Mandatory Quarantine for Travelers_Lifted,Mandatory Quarantine for Travelers_Rolled Back to Certain States,Mandatory Quarantine for Travelers_Rolled Back to International Travel,Mandatory Quarantine for Travelers_unknown,Non-Essential Business Closures_-,Non-Essential Business Closures_All Non-Essential Businesses Open,Non-Essential Business Closures_All Non-Essential Businesses Open with Limits,Non-Essential Business Closures_All Non-Essential Businesses Permitted to Reopen,Non-Essential Business Closures_All Non-Essential Businesses Permitted to Reopen,Non-Essential Business Closures_All Non-Essential Businesses Permitted to Reopen with Reduced Capacity,Non-Essential Business Closures_New Business Closures or Limits,Non-Essential Business Closures_Some Non-Essential Businesses Open with Limits,Non-Essential Business Closures_Some Non-Essential Businesses Permitted to Reopen,Non-Essential Business Closures_Some Non-Essential Businesses Permitted to Reopen with Reduced Capacity,Non-Essential Business Closures_unknown,Large Gatherings Ban_-,Large Gatherings Ban_>10 People Prohibited,Large Gatherings Ban_>10 Prohibited,Large Gatherings Ban_>25 Prohibited,Large Gatherings Ban_>50 Prohibited,Large Gatherings Ban_All Gatherings Prohibited,Large Gatherings Ban_Expanded Limit to 25 or Fewer,Large Gatherings Ban_Expanded Limit to Greater Than 25,Large Gatherings Ban_Expanded to New Limit Above 25,Large Gatherings Ban_Expanded to New Limit Below 25,Large Gatherings Ban_Expanded to New Limit of 25,Large Gatherings Ban_Expanded to New Limit of 25 or Fewer,Large Gatherings Ban_Lifted,Large Gatherings Ban_Limit > 50,Large Gatherings Ban_Limit >50,Large Gatherings Ban_New Limit on Large Gatherings in Place,Large Gatherings Ban_No Limit,Large Gatherings Ban_Other,Large Gatherings Ban_unknown,School Closures_Closed,School Closures_Closed for School Year,School Closures_Recommended Closure,School Closures_Recommended Closure for School Year,School Closures_Rescinded,School Closures_unknown,Restaurant Limits_-,Restaurant Limits_Closed Except for Takeout/Delivery,Restaurant Limits_Closed to Indoor Service,Restaurant Limits_Limited Dine-in Service,Restaurant Limits_New Capacity Limits,Restaurant Limits_New Service Limits,Restaurant Limits_Newly Closed to Dine-in Service,Restaurant Limits_Newly Closed to Indoor Dining,Restaurant Limits_Open,Restaurant Limits_Open with Service Limits,Restaurant Limits_Reopened to Dine-in Service,Restaurant Limits_Reopened to Dine-in Service,Restaurant Limits_Reopened to Dine-in Service with Capacity Limits,Restaurant Limits_unknown,Primary Election Postponement_-,Primary Election Postponement_Canceled,Primary Election Postponement_Postponed,Primary Election Postponement_unknown,Emergency Declaration_No,Emergency Declaration_Yes,Emergency Declaration_unknown,Status of Reopening_Easing Restrictions,Status of Reopening_New Restrictions Imposed,Status of Reopening_Paused,Status of Reopening_Proceeding with Reopening,Status of Reopening_Reopened,Status of Reopening_unknown,Bar Closures_Closed,Bar Closures_New Service Limits,Bar Closures_Newly Closed,Bar Closur

In [56]:
merged_policy.reset_index(drop=True, inplace=True)

In [64]:
merged_policy.shape

(1472417, 338)

In [69]:
pa_dummies.reset_index(drop=True, inplace=True)

In [70]:
merged_policy_dummies = pd.concat([merged_policy, pa_dummies], axis=1)

In [72]:
drop_cols = merged_policy_dummies.loc[:, 'state':'Statewide Face Mask Requirement'].columns

In [73]:
merged_policy_dummies.drop(drop_cols, axis=1, inplace=True)

In [74]:
merged_policy_dummies.info(max_cols=500)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472417 entries, 0 to 1472416
Data columns (total 437 columns):
 #   Column                                                                                                          Non-Null Count    Dtype         
---  ------                                                                                                          --------------    -----         
 0   date                                                                                                            1472417 non-null  datetime64[ns]
 1   state_fips                                                                                                      1472417 non-null  object        
 2   fips                                                                                                            1472417 non-null  int64         
 3   cases_avg                                                                                                       1472417 non-null  f

In [75]:
merged_policy_dummies.to_parquet('/content/drive/MyDrive/social_unrest_data/main_data_policy_text_embeddings_11_15_21.parquet')